In [2]:
from skimage.transform import rotate
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from datetime import datetime
from IPython.display import display, clear_output, Markdown
from PIL import Image
import pandas as pd
from google.colab import files
import zipfile
import ipywidgets as widgets

PASTA_SAIDA = "imagens_processadas"
os.makedirs(PASTA_SAIDA, exist_ok=True)

historico = []
imagens = []

doc_md = """
# Módulo: Redimensionamento / Rotação / Auto-align / Miniaturas

**O que este notebook fornece (resumo):**
- Upload de imagens pelo Colab.
- Pré-visualização em tempo real das transformações.
- Redimensionamento manual/automático (preserva proporção).
- Rotação manual (ângulo) e rotação automática (estimativa via HoughLinesP).
- Recorte automático usando detecção de contornos (ROI).
- `auto_align`: recorte + correção de inclinação (pipeline automático).
- Geração automática de miniaturas (vários tamanhos) e salvamento.
- Histórico e exportação ZIP dos resultados.

Use a interface abaixo para carregar suas imagens, ajustar parâmetros e confirmar o processamento.
"""
display(Markdown(doc_md))

def upload_imagens_colab():
    """Upload de imagens no Colab e retorna lista de (caminho, imagem)."""
    uploaded = files.upload()
    imgs = []
    for nome in uploaded.keys():
        caminho = nome
        img = cv2.imread(caminho)
        if img is not None:
            imgs.append((caminho, img))
        else:
            print(f"Erro ao carregar {caminho}")
    return imgs

def validar_imagem(img):
    return img is not None and img.shape[0] > 10 and img.shape[1]

def redimensionar_imagem(img, largura_alvo=None, altura_alvo=None):
    """Redimensiona mantendo proporção aproximada."""
    h, w = img.shape[:2]
    if largura_alvo and altura_alvo:
        proporcao = min(largura_alvo / w, altura_alvo / h)
        novo_w = int(w * proporcao)
        novo_h = int(h * proporcao)
    elif largura_alvo:
        proporcao = largura_alvo / w
        novo_w = largura_alvo
        novo_h = int(h * proporcao)
    elif altura_alvo:
        proporcao = altura_alvo / h
        novo_w = int(w * proporcao)
        novo_h = altura_alvo
    else:
        return img
    return cv2.resize(img, (novo_w, novo_h), interpolation=cv2.INTER_CUBIC)

def redimensionar_automatico(img, largura_max=800, altura_max=800):
    """Redimensiona automaticamente mantendo proporção."""
    h, w = img.shape[:2]
    proporcao = min(largura_max / w, altura_max / h, 1.0)
    novo_w, novo_h = int(w * proporcao), int(h * proporcao)
    return cv2.resize(img, (novo_w, novo_h), interpolation=cv2.INTER_CUBIC)


# Módulo: Redimensionamento / Rotação / Auto-align / Miniaturas

**O que este notebook fornece (resumo):**
- Upload de imagens pelo Colab.
- Pré-visualização em tempo real das transformações.
- Redimensionamento manual/automático (preserva proporção).
- Rotação manual (ângulo) e rotação automática (estimativa via HoughLinesP).
- Recorte automático usando detecção de contornos (ROI).
- `auto_align`: recorte + correção de inclinação (pipeline automático).
- Geração automática de miniaturas (vários tamanhos) e salvamento.
- Histórico e exportação ZIP dos resultados.

Use a interface abaixo para carregar suas imagens, ajustar parâmetros e confirmar o processamento.
